# Crowling Berita

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid # Meskipun tidak digunakan untuk ID dari web, bisa untuk ID unik jika diperlukan

In [2]:
# URL homepage CNN Indonesia
url = "https://www.cnnindonesia.com/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0"}

# Ambil halaman utama
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

data = []

# Loop semua link artikel di homepage
for artikel in soup.select("article a"):
    link = artikel.get("href")
    judul = artikel.get_text(strip=True)

    # Skip jika link kosong atau bukan http
    if not link or not link.startswith("http"):
        continue

    try:
        # Ambil halaman detail berita
        resp_detail = requests.get(link, headers=headers)
        soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

        # --- PERUBAHAN DIMULAI DI SINI ---
        
        # Ambil ID Berita dari link
        id_berita = None
        try:
            # Contoh path: /ekonomi/20250910153012-92-1234567/nama-artikel
            # Kita ambil bagian sebelum nama artikel, yaitu '20250910153012-92-1234567'
            id_segment = link.split("/")[-2]
            # Kemudian kita pisah dengan '-' dan ambil bagian terakhirnya
            id_berita = id_segment.split("-")[-1]
        except (IndexError, AttributeError):
            # Jika struktur URL berbeda dan gagal, ID akan tetap None
            id_berita = None

        # Ambil kategori dari link (path pertama setelah domain)
        kategori = None
        try:
            path = link.replace("https://www.cnnindonesia.com/", "")
            kategori = path.split("/")[0] if path else None
        except Exception:
            kategori = None

        # Ambil isi berita
        paragraf = [p.get_text(strip=True) for p in soup_detail.select("div.detail-text p")]
        isi = " ".join(paragraf)

        if isi:  # hanya simpan kalau ada isi berita
            data.append({
                "id_berita": id_berita, # <-- Kolom baru ditambahkan di sini
                "judul": judul,
                "kategori": kategori,
                "isi": isi,
                "link": link
            })
            
        # --- PERUBAHAN SELESAI DI SINI ---

    except Exception as e:
        print(f"Gagal ambil {link}: {e}")

# Simpan ke dataframe Pandas
df = pd.DataFrame(data)

# Simpan ke file CSV
df.to_csv("berita_cnn_dengan_id.csv", index=False, encoding="utf-8-sig")

# Tampilkan hasil di notebook
pd.set_option("display.max_colwidth", 100)  # biar isi tidak kepotong
print("Proses selesai. Berikut adalah contoh datanya:")
display(df)

Proses selesai. Berikut adalah contoh datanya:


,id_berita,judul,kategori,isi,link
0,1279939,Mahkamah PPP Bantah Ada Isu Dua Dualisme Usai Muktamar,nasional,Mahkamah Partai Persatuan Pembangunan (PPP) membantah ada isu dualisme kepemimpinan usai pelaksa...,https://www.cnnindonesia.com/nasional/20251002022400-32-1279939/mahkamah-ppp-bantah-ada-isu-dua-...
1,1279844,PPP Kubu Agus Suparmanto Serahkan Hasil Muktamar X ke Kemenkum,nasional,Partai Persatuan Pembangunan (PPP) kubu Agus Suparmanto menyerahkan kepengurusan hasil Muktamar ...,https://www.cnnindonesia.com/nasional/20251001172701-32-1279844/ppp-kubu-agus-suparmanto-serahka...
2,1279979,01VIDEO: Detik-detik Armada Global Sumud Flotilla Dicegat IsraelInternasional,internasional,"Armada kapalGlobal Sumud Flotilla(GSF) yang membawa bantuan ke Gaza,PalestinadicegatIsraelpada R...",https://www.cnnindonesia.com/internasional/20251002071111-124-1279979/video-detik-detik-armada-g...
3,1279983,"02Global Sumud Flotilla Dicegat IDF, Kolombia Usir Semua Diplomat IsraelInternasional",internasional,Armada Global Sumud Flotilla (GSF) diculikIsraeldi perairan internasional menuju pantai Jalur Ga...,https://www.cnnindonesia.com/internasional/20251002073815-120-1279983/global-sumud-flotilla-dice...
4,1279984,03Netizen Berdebat Usai Daftar 32 Pemain Timnas U-23 SEA Games RilisOlahraga,olahraga,"Netizen riuh usai daftar 32 pemainTimnas Indonesia U-23untuk SEA Games diumumkan. Persoalannya, ...",https://www.cnnindonesia.com/olahraga/20251002073902-142-1279984/netizen-berdebat-usai-daftar-32...
5,1279907,"04Didatangi Basuki, Purbaya Beri 3 Dukungan Pendanaan Buat IKNEkonomi",ekonomi,Menteri Keuangan (Menkeu)Purbaya Yudhi Sadewaberjanji memberi 3 dukungan pendanaan untukIKNNusan...,https://www.cnnindonesia.com/ekonomi/20251001210351-532-1279907/didatangi-basuki-purbaya-beri-3-...
6,1280015,05FIFA Akhirnya Bicara soal Desakan Boikot Israel dari Piala Dunia 2026Olahraga,olahraga,FIFAakhirnya menanggapi seruan agar Israel dilarang tampil di Piala Dunia 2026 yang akan berlang...,https://www.cnnindonesia.com/olahraga/20251002084428-142-1280015/fifa-akhirnya-bicara-soal-desak...
7,1280007,"06Global Sumud Flotilla Dicegat Israel, 2 Kapal Lolos & Lanjut ke GazaInternasional",internasional,ArmadaGlobal Sumud Flotilla(GSF) untuk bantuan kemanusiaan diGazayang terdiri dari 45 kapal maup...,https://www.cnnindonesia.com/internasional/20251002082135-120-1280007/global-sumud-flotilla-dice...
8,1280040,"Jane Goodall Wafat, Pembuka Tabir Dunia Kera hingga Imaji soal Tarzan",internasional,"Ilmuwan dan aktivis pelindung satwa primataJane Goodallmeninggal dunia di usia 91 tahun, Rabu (1...",https://www.cnnindonesia.com/internasional/20251002095458-134-1280040/jane-goodall-wafat-pembuka...
9,1279907,"Didatangi Basuki, Purbaya Beri 3 Dukungan Pendanaan Buat IKN",ekonomi,Menteri Keuangan (Menkeu)Purbaya Yudhi Sadewaberjanji memberi 3 dukungan pendanaan untukIKNNusan...,https://www.cnnindonesia.com/ekonomi/20251001210351-532-1279907/didatangi-basuki-purbaya-beri-3-...
